In [10]:
import pandas as pd
import re

In [11]:
START_LINE = 20996
CURR_DATE = '241014'

In [12]:
file = 'import.xlsx'

In [13]:
df = pd.read_excel(file, dtype={'Strona transakcji': str})

In [14]:
def get_acc_number(rachunek):
    mapping = {
        '2248': '001', '2475': '002', '1123': '003', '5899': '004', '1107': '005',
        '2497': '006', '2516': '007', '0668': '009', '5898': '010', '7200': '011',
        '0460': '012', '2474': '013', '6916': '018', '5195': '019', '0002': '020',
        '0001': '021', '0007': '022'
    }
    
    last_four_digits = rachunek[-4:]
    
    code = mapping.get(last_four_digits, 'Unknown')
    
    journal_batch_name = f'IMPORT-{code}'
    bal_account_no = code
    
    return journal_batch_name, bal_account_no

In [15]:
REZ = r'(^|\b|\D|00000)(1( *\d){6})(\D|\b)'

def find_reservation(text):
    rezerwacje = re.findall(REZ, text)
    
    if len(rezerwacje) == 1 or (len(rezerwacje) == 2 and rezerwacje[0][1].replace(' ', '') == rezerwacje[1][1].replace(' ', '')):
        return rezerwacje[0][1].replace(' ', '')
    else:
        return ''

In [16]:
def get_payment_side(row):
    if row['Bal. Account No.'] == '010':
        if row['Reservation No.'] != '':
            if row['Amount'] < 0:
                result = ('Customer', 'Payment')
            result = ('Customer', 'Refund')
        elif row['Amount'] < 0:
                result = ('Vendor', 'Refund')
        else:
            result = ('Vendor', 'Payment')
    elif row['Bal. Account No.'] == '009':
        result = ('G/L Account', 'Payment')
    else:
        if row['Amount'] < 0:
            result = ('Customer', 'Payment')
        else:
            result = ('Vendor', 'Payment')
    return result

In [17]:
df2 = pd.DataFrame(index=df.index, columns=[
    'Journal Template Name', 'Journal Batch Name', 'Line No.', 'Posting Date', 
    'Document Type', 'Account Type', 'Document No.', 'External Document No.', 
    'Reservation No.', 'Amount', 'Message to Recipient', 'Payer Information', 
    'Bal. Account Type', 'Bal. Account No.'
])
df2['Journal Template Name'] = 'PŁATNOŚCI'
df2['Journal Batch Name'] = df['Rachunek'].apply(lambda x: get_acc_number(x)[0])
df2['Line No.'] = [START_LINE + i for i in range(len(df))]
df2['Posting Date'] = df['Data księgowania']
df2['Document No.'] = [f'imp/{CURR_DATE}/{i:04d}' for i in range(len(df))]
df2['External Document No.'] = df2['Document No.']
df2['Reservation No.'] = df['Tytuł'].apply(find_reservation).replace(' ', '')
df2['Amount'] = -df['Kwota']
df2['Message to Recipient'] = df['Tytuł']
df2['Payer Information'] = df['Strona transakcji'].apply(lambda x: x[:50] if isinstance(x, str) else ' ')
df2['Bal. Account Type'] = 'Bank Account'
df2['Bal. Account No.'] = df['Rachunek'].apply(lambda x: get_acc_number(x)[1])
df2[['Account Type', 'Document Type']] = df2.apply(lambda row: pd.Series(get_payment_side(row)), axis=1)

In [18]:
df2.to_excel('import_output.xlsx', index=False)